# Topic Explainer

In [ ]:
!pip install -q anthropic

In [ ]:
import anthropic
import openai
from openai import OpenAI

In [ ]:
from google.colab import userdata
openai = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
claude = anthropic.Anthropic(api_key= userdata.get('ANTHROPIC_API_KEY'))

In [ ]:
system_message = "You are a well known teacher who is known to explain any topic in very simple and structured way , you first give a 2-3 line introduction about the topic , then you give some important bullet points related to it then\
followed by a simple and short example related to that topic"

In [ ]:
system_message += "/n The user tells you which class they study in along with the topic , and then explain the topic in deapth according to their level of knowledge , example a student can be of class 8,9,10,11,12 or in graduation , or maybe doing a PhD"

In [ ]:
system_message += "/n Make sure the language is very simple and avoid using complex terms where-ever possible"

In [ ]:
system_message += "\n always respond in markdown"

In [ ]:
def stream_gpt(prompt):
    message = [
        {"role" : "system" , "content" : system_message},
        {"role" : "user" , "content" : prompt}
    ]
    response = openai.chat.completions.create(
        model = 'gpt-4o',
        messages = message,
        stream = True
    )
    result = ""
    for chunk in response:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
def stream_claude(prompt):
    response = claude.messages.stream(
        model = 'claude-3-5-sonnet-20240620',
        max_tokens = 1000,
        system = system_message,
        messages = [
            {"role" : "user" , "content" : prompt}
        ]
    )
    result = ""
    with response as stream:
        for text in stream.text_stream:
            result += text or ""
            yield result

In [ ]:
def stream_model(prompt, model):
    if model == "GPT":
        result = stream_gpt(prompt)
    elif model == "Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown Model")

    for chunk in result:
        yield chunk

# ChatBot for Q&A

In [ ]:
system_message2 = "system_message = You are a well known teacher , when someone gives you a topic you give them 5 MCQ questions one by one , the next question appears after the user have answered the previous \
one , and at the end you give score out of 5 and teach them in short and in easy terms what they did not know about the topic (based on their answers)"

In [ ]:
system_message2 += "\n your main focus is on Indian High school subjects like science (physics  , chemistry , biology), maths , history , etc"

In [ ]:
system_message2 += "\n in start you just ask in which class do you study ? n, Tell me the topic ? , just like that short and crisp , then questions accordingly (difficulty/deapth of topic according to\
the class they study in)"

In [ ]:
def gpt_qna(message, history):
    messages = [{"role" : "system" , "content" : system_message2}]
    for user_message , assistant_message in history:
        messages.append({"role" : "user" , "content" : user_message})
        messages.append({"role" : "assistant" , "content" : assistant_message})

    messages.append({"role" : "user" , "content" : message})

    stream = openai.chat.completions.create(model = 'gpt-4o-mini' , messages = messages, stream = True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
def claude_qna(message, history):
    messages = []
    for user_message , assistant_message in history:
        messages.append({"role" : "user" , "content" : user_message})
        messages.append({"role" : "assistant" , "content" : assistant_message})

    messages.append({"role" : "user" , "content" : message})

    response = claude.messages.stream(
        model = 'claude-3-5-sonnet-20240620',
        max_tokens = 400,
        system = system_message2,
        messages = messages
    )
    result = ""
    with response as stream:
        for text in stream.text_stream:
            result += text or ""
            yield result

# Image Generator for explanation
### found two models : Recraft and Flux 1.1 pro

In [ ]:
# import base64
# from io import BytesIO
# from PIL import Image

In [ ]:
# def concept_image(city):
#     image_response = openai.images.generate(
#         model = "dall-e-3",
#         prompt = f"Generate an image related to city {city}  , the image should be such that it might help visualise the person reading about{city}",
#         size = "1024x1024",
#         n=1,
#         response_format = 'b64_json'
#     )
#     image_base64 = image_response.data[0].b64_json
#     image_data = base64.b64decode(image_base64)
#     return Image.open(BytesIO(image_data))

# Pdf to Text

In [ ]:
# # app.py
# import gradio as gr
# from pdf_processing import extract_text_from_pdf
# from embeddings import initialize_faiss, is_faiss_initialized
# from qa_model import retrieve_answer

# def process_pdf_and_answer(pdf, question):
#     # Extract text from PDF and initialize FAISS index
#     text = extract_text_from_pdf(pdf)
#     initialize_faiss(text)

#     if not is_faiss_initialized():
#         return "Error: FAISS index failed to initialize."
#     # Answer the question using retrieval and generation
#     answer = retrieve_answer(question)
#     return answer

# # Define Gradio Inputs and Outputs
# pdf_input = gr.File(label="Upload PDF")
# question_input = gr.Textbox(label="Ask a question about the PDF content")
# output_text = gr.Textbox(label="Answer")

# # Create Gradio Interface
# interface = gr.Interface(
#     fn=process_pdf_and_answer,
#     inputs=[pdf_input, question_input],
#     outputs=output_text,
#     title="COGNIFY PDF ANALYZER",
#     description="Upload a PDF, and ask questions about its content.",
# )

# # Launch the Gradio App
# if __name__ == "__main__":
#     interface.launch(share=True)


ModuleNotFoundError: No module named 'pdf_processing'

In [ ]:
# # qa_model.py
# from embeddings import faiss_index, is_faiss_initialized
# from langchain.chains import RetrievalQA
# from langchain_huggingface import HuggingFacePipeline
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# # Initialize Hugging Face Model for Answer Generation
# def load_qa_model():
#     """
#     Loads a Hugging Face pipeline for text generation and wraps it in LangChain.
#     """
#     model_name = 'meta-llama/Llama-3.1-8B'  # Replace with your Hugging Face model
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModelForCausalLM.from_pretrained(model_name)

#     # Create a text-generation pipeline
#     hf_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer)

#     # Wrap the pipeline in LangChain's HuggingFacePipeline
#     return HuggingFacePipeline(pipeline=hf_pipeline)

# # Load the QA model
# qa_model = load_qa_model()

# def retrieve_answer(question):
#     """
#     Retrieves an answer for the given question using the FAISS index.

#     Args:
#         question (str): The question to ask.

#     Returns:
#         str: The answer retrieved or an error message.
#     """
#     global faiss_index

#     # Check if FAISS index is initialized
#     if not is_faiss_initialized():
#         return "Error: FAISS index is not initialized."

#     try:
#         retriever = faiss_index.as_retriever()
#         qa_chain = RetrievalQA.from_chain_type(llm=qa_model, retriever=retriever)
#         response = qa_chain.run(question)
#         return response
#     except Exception as e:
#         return f"Error during QA retrieval: {e}"


In [ ]:
# # pdf_processing.py
# from PyPDF2 import PdfReader

# def extract_text_from_pdf(pdf):
#     text = ""
#     pdf_reader = PdfReader(pdf)
#     for page in pdf_reader.pages:
#         text += page.extract_text()
#     return text


In [ ]:
# # embeddings.py
# from sentence_transformers import SentenceTransformer
# from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain.docstore.document import Document

# # Global FAISS index
# faiss_index = None

# def initialize_faiss(text):
#     """
#     Initializes a FAISS index with the given text.

#     Args:
#         text (str): The text to embed and index.
#     """
#     global faiss_index

#     if not text:
#         print("Error: No text provided to initialize FAISS index.")
#         return

#     # Initialize Sentence Transformer and HuggingFace embeddings
#     hf_embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
#     documents = [Document(page_content=text)]  # Wrap text in LangChain Document

#     # Create the FAISS index
#     faiss_index = FAISS.from_documents(documents, hf_embeddings)
#     print("FAISS index initialized successfully.")

# def is_faiss_initialized():
#     """
#     Checks if the FAISS index is initialized.

#     Returns:
#         bool: True if the FAISS index is initialized, False otherwise.
#     """
#     global faiss_index
#     return faiss_index is not None

# def query_faiss(query_text):
#     """
#     Queries the FAISS index if it's initialized.

#     Args:
#         query_text (str): The text to query in the FAISS index.

#     Returns:
#         list: The query results if the index is initialized, otherwise an error message.
#     """
#     global faiss_index
#     if not is_faiss_initialized():
#         print("Error: FAISS index is not initialized. Please upload a PDF or text first.")
#         return None

#     # Assuming FAISS index supports a 'similarity_search' method
#     results = faiss_index.similarity_search(query_text)
#     return results


# Image to text

In [ ]:
!pip install paddleocr
!pip install paddlepaddle


In [ ]:
import gradio as gr
from paddleocr import PaddleOCR

def ocr_with_paddle(img):
    finaltext = ''
    ocr = PaddleOCR(lang='en', use_angle_cls=False)
    result = ocr.ocr(img)

    for i in range(len(result[0])):
        text = result[0][i][1][0]
        finaltext += ' ' + text
    return finaltext

def message_for(question):
    return [
        {"role" : "system" , "content" : system_message},
        {"role" : "user" , "content" : f"can you answer the following question in easy terms : {question}"}
    ]
def ask_gpt(question):
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini' ,
        messages = message_for(question)
    )
    return response.choices[0].message.content


def process_image_and_answer(img):
    extracted_text = ocr_with_paddle(img)
    gpt_answer = ask_gpt(extracted_text)
    return gpt_answer


# Text Summarizer

# UI work (Gradio)

In [ ]:
!pip install -q gradio
import gradio as gr

In [ ]:
with gr.Blocks() as demo:

    with gr.Row():
        gr.Image("logonew.png", elem_id="logo", label=None, height=150 )


    with gr.Row():
        with gr.Column():
            topic_input = gr.Textbox(label="Enter your Topic:", placeholder="Type your topic here...")
            model_select = gr.Dropdown(["GPT", "Claude"], label="Select Model")
            topic_output = gr.Markdown(label="Topic Explanation")
            explain_button = gr.Button("Explain Topic")

        explain_button.click(fn=stream_model, inputs=[topic_input, model_select], outputs=topic_output)


        with gr.Column():
            custom_chatbot = gr.Chatbot(height=600)
            chat_interface = gr.ChatInterface(fn=gpt_qna, chatbot=custom_chatbot)


    with gr.Row():
        gr.Button("Clear").click(lambda: None, None, [topic_output])

    with gr.Row():
        with gr.Column():
            gr.Markdown("### Upload a Question Image for AI Explanation")
            image_input = gr.Image(label="Upload Question Image")
            gpt_answer_output = gr.Markdown(label="Answer from ChatGPT")
            process_button = gr.Button("Process Image")

        process_button.click(fn=process_image_and_answer, inputs=image_input, outputs=gpt_answer_output)


    with gr.Row():
        gr.Button("Clear").click(lambda: None, None, [topic_output, gpt_answer_output])


demo.launch(debug = True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://98868a80f9a91eb06f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[2024/11/21 06:46:47] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

In [ ]:
# with gr.Blocks() as demo:
#     # Row for logo
#     with gr.Row():
#         gr.Image("logonew.png", elem_id="logo", label=None, height=150 )

#     # First Column for Topic Explainer
#     with gr.Row():
#         with gr.Column():
#             topic_input = gr.Textbox(label="Enter your Topic:", placeholder="Type your topic here...")
#             model_select = gr.Dropdown(["GPT", "Claude"], label="Select Model")
#             topic_output = gr.Markdown(label="Topic Explanation")
#             explain_button = gr.Button("Explain Topic")

#         explain_button.click(fn=stream_model, inputs=[topic_input, model_select], outputs=topic_output)

#         # Second Column for QnA Chatbot
#         with gr.Column():
#             custom_chatbot = gr.Chatbot(height=600)
#             chat_interface = gr.ChatInterface(fn=gpt_qna, chatbot=custom_chatbot)

#     # Clear Button
#     with gr.Row():
#         gr.Button("Clear").click(lambda: None, None, [topic_output])

# demo.launch(debug = True)
